In [2]:
import re
import string
import pandas as pd
from functools import reduce
from math import log

## Simple example of [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
1. Example of corpus
2. Preprocessing and Tokenizing
3. Calculating bag of words
4. TF
5. IDF
6. TF-IDF

In [3]:
#1
corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
""".split("\n")[1:-1]

In [4]:
#2
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

['simple', 'example', 'with', 'cats', 'and', 'mouse']
['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats']
['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


In [5]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'with', 'another', 'cheese', 'example', 'dogs', 'simple', 'mouse', 'and', 'cats'}


In [6]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,and,another,cats,cheese,dogs,example,mouse,simple,with
0,1,0,1,0,0,1,1,1,1
1,1,1,1,0,1,1,0,1,1
2,1,1,0,1,0,1,1,1,1


词频就是一个词（term）在一个文档中（document）出现的次数（frequency），记为 tf(t,d)。这是一种最简单的定义方式。
词频的优点是简单，但缺点也很显然：
词频中没有包含词的位置信息，所以从词频的角度来看，"Mary is quicker than John"和"John is quicker than Mary"两条文档是完全一致的，但显然它们的含义是完全相反的。
词频没有考虑不同词的重要性一般是不一样的，比如停用词的词频都很高，但它们并不重要。

In [8]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [9]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

In [10]:
#Simple example with Cats and Mouse
tf_A

{'and': 0.16666666666666666,
 'another': 0.0,
 'cats': 0.16666666666666666,
 'cheese': 0.0,
 'dogs': 0.0,
 'example': 0.16666666666666666,
 'mouse': 0.16666666666666666,
 'simple': 0.16666666666666666,
 'with': 0.16666666666666666}

## \#5 idf - inverse document frequency
idf is a measure of how much information the word provides
$$ \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$
- $N$: total number of strings in the corpus ${\displaystyle N={|D|}}$
- ${\displaystyle |\{d\in D:t\in d\}|}$  : number of strings where the term ${\displaystyle t}$ appears (i.e., ${\displaystyle \mathrm {tf} (t,d)\neq 0})$. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to ${\displaystyle 1+|\{d\in D:t\in d\}|}$.

对于固定的语料库，N是固定的，一个词的在各个文献中出现的次数越多，其idf(t,D)就越小。所以那些很稀少的词的idf 值会很高，而像停用词这种出现频率很高的词idf值很低。

In [11]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [12]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
idf

{'and': 0.0,
 'another': 0.4054651081081644,
 'cats': 0.4054651081081644,
 'cheese': 1.0986122886681098,
 'dogs': 1.0986122886681098,
 'example': 0.0,
 'mouse': 0.4054651081081644,
 'simple': 0.0,
 'with': 0.0}

## \# 6 tf-idf
Then tf–idf is calculated as
$$ {\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\cdot \mathrm {idf} (t,D)} $$

TF-IDF就是将TF和IDF结合起来，就是简单地相乘。从公式可以分析出来，一个词 t 在某个文档d 中的tf-idf值：

当该词在少数文档中出现很多次的时候，其值接近最大值；（tf和idf都很大）
当该词在文档中出现次数少或者在很多文档中都出现时，其值较小；（tf或idf比较小）
当该词几乎在所有文档中都出现时，其值接近最小值。（idf很小，接近0）

In [10]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [11]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [12]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,and,another,cats,cheese,dogs,example,mouse,simple,with
0,0.0,0.000000,0.067578,0.000000,0.000000,0.0,0.067578,0.0,0.0
1,0.0,0.057924,0.057924,0.000000,0.156945,0.0,0.000000,0.0,0.0
2,0.0,0.057924,0.000000,0.156945,0.000000,0.0,0.057924,0.0,0.0


# For clustering we must use tf-idf weights
the example above is just an example, in practice it is better to apply [TfidfVectorizer from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Full text for clusterring

This corpus contain some strings about Google and some strings about TF-IDF from Wikipedia. Just for example

In [7]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
Taylor Alison Swift is an American singer-songwriter. 
Taylor is known for narrative songs about her personal life
Her songs have received widespread media coverage.
Her album won four Grammy Awards.
Swift is the youngest Grammy Album of the Year winner.
""".split("\n")[1:-1]

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

In [20]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

Now, let's calculate tf-idf for this corpus

In [21]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)

And train simple kmeans model with k = 3

In [22]:
kmeans = KMeans(n_clusters=3).fit(tfidf)

Predictions

In [24]:
lines_for_predicting = ["tf and idf is awesome!", "Google's android system is widely used.","Taylor is a young idol."]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([1, 0, 2])